In [1]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

In [2]:
#Import all snowflake connection details from template variables.
db_user = 'MANISH'
db_password = 'Password@2023'
db_account = 'ug94937.us-east4.gcp'
db_database =  'FDC_BANKING_FS'
db_role = 'MANISH'
db_warehouse = 'FOSFOR_SOLUTIONS_WH'
db_schema = 'PUBLIC'

In [3]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session1 = Session.builder.configs(connection_params).create()

In [4]:
session1.sql('use warehouse FOSFOR_SOLUTIONS_WH;').collect()
session1.sql('use database FDC_BANKING_FS;').collect()
session1.sql('use schema FDC_BANKING_FS.PUBLIC;').collect()

[Row(status='Statement executed successfully.')]

In [5]:
atm = pd.read_csv('/data/atms_data.csv')

In [6]:
atm.shape

(420, 4)

In [31]:
atm.columns = ['atm_id', 'max_capacity', 'bank_id', 'state']

In [32]:
atm.head()

,atm_id,max_capacity,bank_id,state
0,SPCN2020,2640000.0,1,Michigan
1,SPCN2021,2640000.0,1,Florida
2,SPCN2022,2640000.0,1,Nebraska
3,SPCN2023,2640000.0,1,Kansas
4,SPCN2024,2640000.0,1,New Hampshire


In [33]:
atm.isnull().sum()

atm_id          0
max_capacity    0
bank_id         0
state           0
dtype: int64

In [34]:
sf_atm=session1.createDataFrame(
        atm.values.tolist(),
        schema=atm.columns.tolist())
sf_atm.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.ATM_MASTER")

In [10]:
banks = pd.read_csv('/data/banks_data.csv')

In [36]:
banks.columns = ['bank_id', 'bank_name']

In [37]:
banks.head()

,bank_id,bank_name
0,1,Wells Fargo
1,2,BNY Mellon
2,3,Citibank


In [38]:
banks.shape

(3, 2)

In [39]:
banks.isnull().sum()

bank_id      0
bank_name    0
dtype: int64

In [40]:
sf_banks=session1.createDataFrame(
        banks.values.tolist(),
        schema=banks.columns.tolist())
sf_banks.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.BANK_MASTER")

In [16]:
day_data = pd.read_csv('/data/day_data.csv')

In [17]:
day_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   caldate     783 non-null    object
 1   weekday     783 non-null    object
 2   holiday     783 non-null    bool  
dtypes: bool(1), object(2)
memory usage: 13.1+ KB


In [20]:
day_data.columns = ['date', 'weekday', 'holiday']

In [21]:
day_data.isnull().sum()

date       0
weekday    0
holiday    0
dtype: int64

In [22]:
sf_day_data=session1.createDataFrame(
        day_data.values.tolist(),
        schema=day_data.columns.tolist())
sf_day_data.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.DATE_MASTER")

In [23]:
transaction = pd.read_csv('/data/transaction_data.csv')

In [26]:
transaction.columns = ['ATMID', 'date', 'Dispense', 'DT', 'CountTotalTxn']

In [27]:
transaction.head()

,ATMID,date,Dispense,DT,CountTotalTxn
0,SPCN2020,2021-01-01,508050.0,0.0,141.0
1,SPCN2020,2021-01-02,467280.0,3.0,117.0
2,SPCN2020,2021-01-03,400140.0,0.0,115.0
3,SPCN2020,2021-01-04,148140.0,151.0,58.0
4,SPCN2020,2021-01-05,360270.0,0.0,98.0


In [28]:
transaction.isnull().sum()

ATMID            0
date             0
Dispense         0
DT               0
CountTotalTxn    0
dtype: int64

In [29]:
sf_transaction=session1.createDataFrame(
        transaction.values.tolist(),
        schema=transaction.columns.tolist())
sf_transaction.write.mode("overwrite").save_as_table("FDC_BANKING_FS.PUBLIC.TRANSACTION_MASTER")